<a href="https://colab.research.google.com/github/Elliot-CRT/103-Final-Game/blob/main/NBA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Get Data

In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import sys
import os
import copy
import numpy as np
import pandas as pd

import joblib

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

# for reproducibility
seed = 23
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

colab = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [2]:
if colab:
    from google.colab import drive
    drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
load_saved_data = True # False
colab_folder_path = "/content/gdrive/MyDrive/NBA/"

# data_file_name_X = "NBA_X.pt"
# data_file_name_y = "NBA_y.pt"
# num_players_per_team = 6
# encoder_name = "nba_encoder.joblib"
# model_name = "nba_model"

# data_file_name_X = "LOL_X.pt"
# data_file_name_y = "LOL_y.pt"
# num_players_per_team = 5
# encoder_name = "lol_encoder.joblib"
# model_name = "lol_model"

data_file_name_X = "amateurX.pt"
data_file_name_y = "amateury.pt"
num_players_per_team = 5
len_aux_column = 10
encoder_name = "amateur_encoder.joblib"
model_name = "amateur_model"

# data_file_name_X = "hero_games_X.pt"
# data_file_name_y = "hero_games_y.pt"
# num_players_per_team = 3
# encoder_name = "hero_games_encoder.joblib"
# model_name = "hero_games_model"

# data_file_name_X = "hero_games_X_10.pt"
# data_file_name_y = "hero_games_y_10.pt"
# num_players_per_team = 10
# encoder_name = "hero_games_encoder_10.joblib"
# model_name = "hero_games_model_10"

# data_file_name_X = "Valorant_X.pt"
# data_file_name_y = "Valorant_y.pt"
# num_players_per_team = 5
# len_aux_column = 40
# encoder_name = "Valorant_encoder.joblib"
# model_name = "Valorant_model"

In [4]:
if(load_saved_data):
    X = torch.load(colab_folder_path + data_file_name_X, weights_only=False)
    true_y = torch.load(colab_folder_path + data_file_name_y, weights_only=False)
    #notice that later when we pass X into model, we will split this into a int comp tensor and a float aux tensor
    # X = torch.tensor(X)
    y = true_y
    # Load the encoder
    encoder = joblib.load(colab_folder_path + encoder_name)

Following is old code that assigns each unique values of X to an unique integer.
This is no longer applicable because we now include aux stats in X, which are meant to be their original float values, not to be converted to integer labels.
Moreover, the integer labeling of comp data is now handled in section "Label Encoding X", which preprocess X anyway.

In [5]:

# if(load_saved_data):
#     ##TODO needs rework for lol
#     X = torch.load(colab_folder_path + data_file_name_X)
#     true_y = torch.load(colab_folder_path + data_file_name_y)
#     X = torch.tensor(X).type(torch.int32)
#     unique_values_in_X = np.unique(X)

#     print("IMPORTANT, certain values missing after downsampling in data preprocessing")
#     print(len(unique_values_in_X))

#     # prompt: which numbers are missing in unique_values_in_X
#     all_possible_numbers = np.arange(np.min(unique_values_in_X), np.max(unique_values_in_X) + 1)
#     missing_numbers = np.setdiff1d(all_possible_numbers, unique_values_in_X)
#     print("Missing numbers:", missing_numbers)

#     # prompt: rearrange values in X so there is no gap
#     mapping = {old_val: new_val for new_val, old_val in enumerate(np.unique(X))}
#     X_rearranged = np.vectorize(mapping.get)(X)

#     print("Original X:", X[:10])
#     print("Rearranged X:", X_rearranged[:10])

#     X = torch.tensor(X_rearranged).type(torch.int32)
#     # y = torch.from_numpy(true_y)
#     y = true_y

#     # prompt: load the encoder

#     # Load the encoder
#     encoder = joblib.load(colab_folder_path + encoder_name)


In [6]:
# prompt: generate statistics about X, how many rows, dimension,

print("Shape of X:", X.shape)
print("Number of rows in X:", X.shape[0])
print("Dimension of X:", X.shape[1])
print("Number of unique values in y:", len(np.unique(y)))

Shape of X: (156348, 20)
Number of rows in X: 156348
Dimension of X: 20
Number of unique values in y: 2


In [7]:
X[0]

array([1.80000e+01, 1.26000e+02, 9.80000e+01, 9.50000e+01, 9.30000e+01,
       6.30000e+01, 2.00000e+00, 8.60000e+01, 1.18000e+02, 1.12000e+02,
       9.69800e+04, 2.59950e+04, 1.71568e+05, 1.13721e+05, 1.85302e+05,
       1.48791e+05, 1.30692e+05, 1.45731e+05, 1.82680e+05, 8.57850e+04])

In [8]:
def prep_player_names(df):
  names = df.values.flatten()
  names = np.unique(names)
  names.sort()
  return names

##Data Checking

In [9]:
def percent_wins(y_list):
  count_ones = sum(1 for outcome in y_list if outcome == 1)

  # Calculate the percentage
  percentage = (count_ones / len(y_list)) * 100

  return percentage
  # print(f"Percentage of tensors with value 1: {percentage:.2f}%")

percent_wins(y)

50.0

#Preprocessing  NBA

IMPORTANT:: If load_saved_data is true, don't need to run this section

Old: Using team information to build our X

In [ ]:
# Example 2D lists
list1 = [
    ['a', 'b', 'c'],
    ['d', 'e', 'f']
]

list2 = [
    ['g', 'h', 'i'],
    ['j', 'k', 'l']
]

# Ensure both lists have the same number of rows
if len(list1) != len(list2):
    raise ValueError("Both 2D lists must have the same number of rows")

# Concatenate the 2D lists column-wise
concatenated_list = [row1 + row2 for row1, row2 in zip(list1, list2)]

# Print the concatenated list
for row in concatenated_list:
    print(row)


['a', 'b', 'c', 'g', 'h', 'i']
['d', 'e', 'f', 'j', 'k', 'l']


In [ ]:
# if(load_saved_data):
#     X = torch.load(colab_folder_path + data_file_name_X)
#     true_y = torch.load(colab_folder_path + data_file_name_y)

# else:
#     colab_folder_path = "/content/gdrive/MyDrive/2023 research/NBA/"
#     games_df = pd.read_csv(colab_folder_path + "NBA_Games_History.csv") #, dtype = str, delimiter = ",", skip_header=1)
#     teams_df = pd.read_csv(colab_folder_path + "NBA_Teams_Training0515.csv") #, dtype = str, delimiter = ",", skip_header=1)
#     #drop irrelevant columns or columns with data leakage issue
#     teams_df = teams_df.drop(columns=['GP', 'MPG'])
#     print(games_df.loc[0]["SEASON"].dtype)
#     teams_df["Season"] = teams_df["Season"].astype(int)
#     list_X = []
#     for index, row in games_df.iterrows():
#         ti1 = teams_df[ (teams_df["Season"]==row['SEASON'] ) &  (teams_df["Team"]==row['HOME_TEAM'] ) ]
#         ti2 = teams_df[ (teams_df["Season"]==row['SEASON'] ) &  (teams_df["Team"]==row['AWAY_TEAM'] ) ]
#         #concatenate the two into x.
#         #drop everything except team
#         ti1 = ti1.drop(columns=['Season', 'Team'])
#         ti1 = torch.tensor(ti1.values)
#         # do the same thing with ti2
#         ti2 = ti2.drop(columns=['Season', 'Team'])
#         ti2 = torch.tensor(ti2.values)
#         # concatenate ti1 and ti2 as a row
#         list_X.append(torch.cat((ti1, ti2), dim = 1))
#     ##return all the rows of concatenated ti1 ti2
#     X = torch.cat(list_X, dim=0)
#     print(X[0])

#     true_y = games_df.drop(columns=['GAME_DATE','HOME_TEAM','AWAY_TEAM','SEASON','PTS_HOME','PTS_AWAY'])
#     # drops all columns but 'HOME_TEAM_WINS'

#     true_y = torch.tensor(true_y.values)

#     # save the X so we don't have to recompute it every time
#     torch.save(X, colab_folder_path + data_file_name_X)
#     # save the y so we don't have to recompute it every time
#     torch.save(true_y, colab_folder_path + data_file_name_y)


New: supposed to work with lineup

In [ ]:
# prompt: create a dictionary that maps the all caps abbreviated NBA city names to their respective capitilized city names. For example "GSW" should map to "Golden State"

nba_teams_short2long_dict = {
    "ATL": "Atlanta",
    "BOS": "Boston",
    "BRK": "Brooklyn",
    "CHA": "Charlotte",
    "CHO": "Charlotte",
    "CHI": "Chicago",
    "CLE": "Cleveland",
    "DAL": "Dallas",
    "DEN": "Denver",
    "DET": "Detroit",
    "GSW": "Golden State",
    "HOU": "Houston",
    "IND": "Indiana",
    "LAC": "L.A. Clippers",
    "LAL": "L.A. Lakers",
    "MEM": "Memphis",
    "MIA": "Miami",
    "MIL": "Milwaukee",
    "MIN": "Minnesota",
    "NOP": "New Orleans",
    "NOH": "New Orleans",
    "NOK": "New Orleans",
    "NYK": "New York",
    "OKC": "Oklahoma City",
    "ORL": "Orlando",
    "PHI": "Philadelphia",
    "PHO": "Phoenix",
    "POR": "Portland",
    "SAC": "Sacramento",
    "SAS": "San Antonio",
    "TOR": "Toronto",
    "UTA": "Utah",
    "WAS": "Washington",
    "NJN": "Brooklyn",
    "SEA": "Oklahoma City"
}


In [ ]:
def prep_city_names(df):

  box_df.iloc[:,1]

In [ ]:
# prompt: write a function that iterates over the second column of each row in a dataframe and converts it from a key in a dictionary to its corresponding value in that dictionary

def convert_dict_values(df, dict_to_use):
  """
  This function iterates over the second column of each row in a dataframe and converts it from a key in a dictionary to its corresponding value in that dictionary.

  Args:
      df: The dataframe to process.
      dict_to_use: The dictionary to use for conversion.

  Returns:
      A new dataframe with the converted values.
  """

  new_df = df.copy()
  for index, row in df.iterrows():
    new_df.iloc[index, 1] = dict_to_use[row[1]]
  return new_df

# Example usage
#new_df = convert_dict_values(box_df, nba_teams_short2long_dict)


In [ ]:
def prep_player_names(box_df):
  names = box_df.iloc[:, 2:].values.flatten()
  names = np.unique(names)
  names.sort()
  return names

In [ ]:
def one_hot_encoding(name, names):
  one_hot_encoding = np.zeros(len(names))
  index = np.where(names == name)[0]
  one_hot_encoding[index] = 1
  return one_hot_encoding


def multi_one_hot_encoding(name, names):
  one_hot_encoding = np.zeros(len(names))
  indices = np.where(np.isin(names, name))[0]
  one_hot_encoding[indices] = 1
  return one_hot_encoding


In [ ]:
box_df = pd.read_csv(colab_folder_path + "/lineup data/combined_data.csv", header=None)
games_df = pd.read_csv(colab_folder_path + "/NBA_Games_History.csv") #, dtype = str, delimiter = ",", skip_header=1)
pattern = r'(?P<month>10)/(?P<day>\d{1,2})/(?P<year>2005)'
games_df = games_df[~games_df['GAME_DATE'].str.contains(pattern)]
box_df.head()

<ipython-input-15-a038cfeb244e>:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  games_df = games_df[~games_df['GAME_DATE'].str.contains(pattern)]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,11/1/2005,PHI,Allen Iverson,Chris Webber,Kyle Korver,Andre Iguodala,Steven Hunter,John Salmons,Michael Redd,Bobby Simmons,Jamaal Magloire,Andrew Bogut,T.J. Ford,Mo Williams
1,11/1/2005,NOK,Chris Paul,P.J. Brown,J.R. Smith,David West,Boštjan Nachbar,Chris Andersen,Peja Stojaković,Shareef Abdur-Rahim,Bonzi Wells,Mike Bibby,Brad Miller,Kevin Martin
2,11/1/2005,SAS,Tim Duncan,Tony Parker,Manu Ginóbili,Bruce Bowen,Rasho Nesterović,Michael Finley,Andre Miller,Voshon Lenard,Carmelo Anthony,Kenyon Martin,Marcus Camby,Earl Boykins
3,11/1/2005,PHO,Raja Bell,Shawn Marion,James Jones,Steve Nash,Kurt Thomas,Jim Jackson,Josh Howard,Dirk Nowitzki,Jason Terry,Doug Christie,Erick Dampier,Keith Van Horn
4,11/2/2005,CLE,LeBron James,Larry Hughes,Drew Gooden,Zydrunas Ilgauskas,Eric Snow,Damon Jones,Chris Paul,P.J. Brown,David West,J.R. Smith,Boštjan Nachbar,Chris Andersen


In [ ]:
# prompt: drop all games in games_df that were played in October. Use a regular expression

#pattern = r'(?P<month>10)/(?P<day>\d{1,2})/(?P<year>2005)'
pattern = r'(^\d{4}-10-\d{2}$)|(^10/\d{2}/\d{4}$)|(^\d{2}-10-\d{4}$)'
test_game_df = games_df[~games_df['GAME_DATE'].str.contains(pattern)]

test_box_df = box_df[~box_df[0].str.contains(pattern)]

print(len(test_game_df))
print(len(test_box_df))

14159
16452


<ipython-input-16-f7d4fe9d8c32>:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  test_game_df = games_df[~games_df['GAME_DATE'].str.contains(pattern)]
<ipython-input-16-f7d4fe9d8c32>:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  test_box_df = box_df[~box_df[0].str.contains(pattern)]


In [ ]:
colab_folder_path = "/content/gdrive/MyDrive/NBA/"
games_df = pd.read_csv(colab_folder_path + "NBA_Games_History.csv") #, dtype = str, delimiter = ",", skip_header=1)

box_df = pd.read_csv(colab_folder_path + "lineup data/combined_data.csv", header=None)
player_names = prep_player_names(box_df)

In [ ]:
len(player_names)

1229

Extract relevant information

In [ ]:
from sklearn.preprocessing import LabelEncoder


if(load_saved_data):
    X = torch.load(colab_folder_path + data_file_name_X)
    true_y = torch.load(colab_folder_path + data_file_name_y)

else:
    colab_folder_path = "/content/gdrive/MyDrive/NBA/"
    games_df = pd.read_csv(colab_folder_path + "NBA_Games_History.csv") #, dtype = str, delimiter = ",", skip_header=1)

    box_df = pd.read_csv(colab_folder_path + "lineup data/combined_data.csv", header=None)
    player_names = prep_player_names(box_df)

    # prompt: convert by box_df so that the second column's values are converted to long names using the dictionary above

    #box_df.iloc[:,1] = box_df.iloc[:,1].map(nba_teams_short2long_dict)

    #drop irrelevant columns or columns with data leakage issue
    pattern = r'(?P<month>10)/(?P<day>\d{1,2})/(?P<year>2005)'
    games_df = games_df[~games_df['GAME_DATE'].str.contains(pattern)]

    games_df.head()


    # print(box_df.iloc[:, 1].unique())
    # for key, value in nba_teams_dict.items():
    #   print(f"Key: {key}, Value: {value}")
    # print(games_df['HOME_TEAM'].unique())


    # converts box_df home_team abbreviations to games_df home_team city name
    unique_box_teams = set()
    for index, row in games_df.iterrows():
      # Check if the value exists as a key in nba_teams_dict
      team_value = nba_teams_short2long_dict.get(box_df.iloc[index, 1])
      if team_value is not None:
          unique_box_teams.add(team_value)

    print(len(unique_box_teams))
    box_df = convert_dict_values(box_df, nba_teams_short2long_dict)

    list_X = []

    # create separate lists for home and away team for Siamese Network training
    list_hometeam = []
    list_awayteam = []

    true_y = games_df.drop(columns=['GAME_DATE','HOME_TEAM','AWAY_TEAM','SEASON','PTS_HOME','PTS_AWAY'])
    # drops all columns but 'HOME_TEAM_WINS'
    true_y = torch.tensor(true_y.values)
    list_true_y = []

    for index, row in games_df.iterrows():
        # print(sum(box_df.iloc[:, 0] == row['GAME_DATE'] )) #"06/19/2016")) #row['GAME_DATE'] ))
        # print(sum(box_df.iloc[:, 1] == row['HOME_TEAM'] )) #nba_teams_dict.get(row['HOME_TEAM']) ))
        box_row = box_df[ (box_df.iloc[:, 0] ==row['GAME_DATE'] ) &
                          (box_df.iloc[:, 1] ==row['HOME_TEAM'] ) ]
        ##bandit fix for now. some game does not have a corresponding box
        ##TODO:: investigate.
        if box_row.shape[0] == 0:
          #TODO debug here
          continue
        #get values from column 2 to column 7 in box_row
        # print("the box_row")
        # print(box_row)
        # print("the box_row end")
        #+ home team, - away team.
        # ti1_one_hot = multi_one_hot_encoding(box_row.iloc[:, 2:8].values, player_names)
        # ti2_one_hot = multi_one_hot_encoding(box_row.iloc[:, 8:].values, player_names)
        ti1_string = box_row.iloc[:, 2:8].values
        ti2_string = box_row.iloc[:, 8:].values


        # Ensure both lists have the same number of rows
        if len(ti1_string) != len(ti2_string):
            raise ValueError("Both 2D lists must have the same number of rows")

        # Concatenate the 2D lists column-wise
        concatenated_array = np.concatenate((ti1_string, ti2_string), axis=1)

        list_X.append(concatenated_array.tolist()[0])
        list_true_y.append(true_y[index])

        # print(box_row.iloc[:, 2:8].values)
        # print(ti1_one_hot)
        # print(box_row.iloc[:, 8:].values)
        # print(ti2_one_hot)
        # ti1 = torch.tensor(ti1_one_hot)
        # ti2 = torch.tensor(ti2_one_hot)

        # list_hometeam.append(ti1)
        # list_awayteam.append(ti2)

        # print(ti1)
        # print(ti2)
        # ti1 = teams_df[ (teams_df["Season"]==row['SEASON'] ) &  (teams_df["Team"]==row['HOME_TEAM'] ) ]
        ## ti2 = teams_df[ (teams_df["Season"]==row['SEASON'] ) &  (teams_df["Team"]==row['AWAY_TEAM'] ) ]
        # bs = box_df[ (box_df.iloc[:, 1]==row['SEASON'] )]
        # #concatenate the two into x.
        # #drop everything except team
        # ti1 = ti1.drop(columns=['Season', 'Team'])
        # ti1 = torch.tensor(ti1.values)
        # # do the same thing with ti2
        # ti2 = ti2.drop(columns=['Season', 'Team'])
        # ti2 = torch.tensor(ti2.values)
        # concatenate ti1 and ti2 as a row
        # list_X.append(torch.cat((ti1, ti2), dim = 1))


        # list_X.append(ti1-ti2)




NameError: name 'load_saved_data' is not defined

In [ ]:
X = list_X
y = list_true_y

NameError: name 'list_X' is not defined

#Preprocessing casual LOL

https://www.kaggle.com/code/adityadesai13/visualisation-analysis-champions-overall-game


the list of champion IDs
https://www.kaggle.com/code/adityadesai13/visualisation-analysis-champions-overall-game/input

In [10]:
colab_path = "/content/gdrive/MyDrive/NBA/Public Games"
filepath = f"{colab_path}/matches.csv"
matches = pd.read_csv(filepath)
filepath = f"{colab_path}/champs.csv"
champs = pd.read_csv(filepath)
filepath = f"{colab_path}/participants.csv"
participants = pd.read_csv(filepath)
filepath = f"{colab_path}/stats1.csv"
stats1 = pd.read_csv(filepath)
filepath = f"{colab_path}/stats2.csv"
stats2 = pd.read_csv(filepath)
stats = pd.concat([stats1, stats2], ignore_index=True)
filepath = f"{colab_path}/teamstats.csv"
teamstats = pd.read_csv(filepath)

/tmp/ipython-input-10-4093558739.py:11: DtypeWarning: Columns (52) have mixed types. Specify dtype option on import or set low_memory=False.
  stats2 = pd.read_csv(filepath)


In [11]:
matches.head()

,id,gameid,platformid,queueid,seasonid,duration,creation,version
0,10,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675
1,11,3187425281,EUW1,420,8,1693,1495066760778,7.10.187.9675
2,12,3187269801,EUW1,420,8,1482,1495053375889,7.10.187.9675
3,13,3187252065,EUW1,420,8,1954,1495050993613,7.10.187.9675
4,14,3187201038,EUW1,420,8,2067,1495047893400,7.10.187.9675


In [12]:
stats.head()

,id,win,item1,item2,item3,item4,item5,item6,trinket,kills,...,neutralminionskilled,ownjunglekills,enemyjunglekills,totcctimedealt,champlvl,pinksbought,wardsbought,wardsplaced,wardskilled,firstblood
0,9,0,3748,2003,3111,3053,1419,1042,3340,6,...,69,42,27,610,13,0,0,10,0,0
1,10,0,2301,3111,3190,3107,0,0,3364,0,...,1,1,0,211,14,1,0,17,3,0
2,11,0,1055,3072,3006,3031,3046,1036,3340,7,...,3,1,2,182,14,1,0,13,5,0
3,12,0,1029,3078,3156,1001,3053,0,3340,5,...,6,6,0,106,15,0,0,3,0,0
4,13,0,3020,1058,3198,3102,1052,1026,3340,2,...,4,3,1,159,15,0,0,10,0,0


In [13]:
df = pd.merge(participants, stats, how = 'left', on = ['id'], suffixes=('', '_y'))
df = pd.merge(df, champs, how = 'left', left_on = 'championid', right_on = 'id', suffixes=('', '_y'))
df = pd.merge(df, matches, how = 'left', left_on = 'matchid', right_on = 'id', suffixes=('', '_y'))
df.head()

,id,matchid,player,championid,ss1,ss2,role,position,win,item1,...,name,id_y,id_y,gameid,platformid,queueid,seasonid,duration,creation,version
0,9,10,1,19,4,11,NONE,JUNGLE,0.0,3748.0,...,Warwick,19,10,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675
1,10,10,2,267,3,4,DUO_SUPPORT,BOT,0.0,2301.0,...,Nami,267,10,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675
2,11,10,3,119,7,4,DUO_CARRY,BOT,0.0,1055.0,...,Draven,119,10,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675
3,12,10,4,114,12,4,SOLO,TOP,0.0,1029.0,...,Fiora,114,10,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675
4,13,10,5,112,4,3,SOLO,MID,0.0,3020.0,...,Viktor,112,10,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675


In [14]:
df.columns

Index(['id', 'matchid', 'player', 'championid', 'ss1', 'ss2', 'role',
       'position', 'win', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6',
       'trinket', 'kills', 'deaths', 'assists', 'largestkillingspree',
       'largestmultikill', 'killingsprees', 'longesttimespentliving',
       'doublekills', 'triplekills', 'quadrakills', 'pentakills',
       'legendarykills', 'totdmgdealt', 'magicdmgdealt', 'physicaldmgdealt',
       'truedmgdealt', 'largestcrit', 'totdmgtochamp', 'magicdmgtochamp',
       'physdmgtochamp', 'truedmgtochamp', 'totheal', 'totunitshealed',
       'dmgselfmit', 'dmgtoobj', 'dmgtoturrets', 'visionscore', 'timecc',
       'totdmgtaken', 'magicdmgtaken', 'physdmgtaken', 'truedmgtaken',
       'goldearned', 'goldspent', 'turretkills', 'inhibkills',
       'totminionskilled', 'neutralminionskilled', 'ownjunglekills',
       'enemyjunglekills', 'totcctimedealt', 'champlvl', 'pinksbought',
       'wardsbought', 'wardsplaced', 'wardskilled', 'firstblood', 'name

In [15]:
stats.columns

Index(['id', 'win', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6',
       'trinket', 'kills', 'deaths', 'assists', 'largestkillingspree',
       'largestmultikill', 'killingsprees', 'longesttimespentliving',
       'doublekills', 'triplekills', 'quadrakills', 'pentakills',
       'legendarykills', 'totdmgdealt', 'magicdmgdealt', 'physicaldmgdealt',
       'truedmgdealt', 'largestcrit', 'totdmgtochamp', 'magicdmgtochamp',
       'physdmgtochamp', 'truedmgtochamp', 'totheal', 'totunitshealed',
       'dmgselfmit', 'dmgtoobj', 'dmgtoturrets', 'visionscore', 'timecc',
       'totdmgtaken', 'magicdmgtaken', 'physdmgtaken', 'truedmgtaken',
       'goldearned', 'goldspent', 'turretkills', 'inhibkills',
       'totminionskilled', 'neutralminionskilled', 'ownjunglekills',
       'enemyjunglekills', 'totcctimedealt', 'champlvl', 'pinksbought',
       'wardsbought', 'wardsplaced', 'wardskilled', 'firstblood'],
      dtype='object')

In [16]:
aux_columns = ['totdmgdealt'] #['kills', 'deaths', 'assists', 'totdmgdealt']
stats[aux_columns].head(10)

,totdmgdealt
0,96980
1,25995
2,171568
3,113721
4,185302
5,148791
6,130692
7,145731
8,182680
9,85785


In [17]:
# prompt: show me all the various values of queueid in df

print(df['queueid'].unique())

[420 440 410   4  42   9  41]


In [18]:
# Explicitly list all columns to keep
columns_to_keep = ["matchid", "championid", "win", "gameid", "queueid"] + aux_columns
df = df[columns_to_keep]
df = df[df["queueid"] == 420]
df = df.drop(columns=["queueid"])

In [19]:
df.head(22)

,matchid,championid,win,gameid,totdmgdealt
0,10,19,0.0,3187427022,96980.0
1,10,267,0.0,3187427022,25995.0
2,10,119,0.0,3187427022,171568.0
3,10,114,0.0,3187427022,113721.0
4,10,112,0.0,3187427022,185302.0
5,10,72,1.0,3187427022,148791.0
6,10,3,1.0,3187427022,130692.0
7,10,103,1.0,3187427022,145731.0
8,10,222,1.0,3187427022,182680.0
9,10,161,1.0,3187427022,85785.0


In [20]:
from sklearn.preprocessing import LabelEncoder

load_saved_data = False
if(not load_saved_data):
  X = []
  y = []
  list_X = []
  prev_row = df.iloc[0]
  prev_match_id = prev_row["matchid"]
  for i, row in df.iterrows():
    match_id = row["matchid"]
    if match_id != prev_match_id:
      if len(list_X) == 2 * num_players_per_team + 2 * num_players_per_team * len(aux_columns):
        # Reorder list_X before appending
        champ_ids = list_X[::(1 + len(aux_columns))]  # every (1 + aux)th element starting from 0
        aux_values = []
        for j in range(len(aux_columns)):
          aux_values += list_X[j + 1::(1 + len(aux_columns))]  # start at j+1, then every step

        listX_reordered = champ_ids + aux_values
        X.append(listX_reordered)
        y.append(prev_row["win"])
      prev_match_id = match_id
      prev_row = row
      list_X = []
    # Add champion ID and aux column values for this player
    list_X.append(row["championid"])
    for col in aux_columns:
      list_X.append(row[col])
  #the above code already reorder this
  #[ champ, dmg, champ, dmg, champ, dmg]
  #into this
  #[champ, champ, champ, dmg,dmg,dmg]

  flattened_data = [item for sublist in X for item in sublist]
  # encoder = LabelEncoder()
  # integer_encoded = encoder.fit_transform(flattened_data)

  num_rows = len(X)
  num_columns = len(X[0])
  flattened_data = np.array(flattened_data).reshape(num_rows, num_columns)

  X = flattened_data
  y = np.array(y)

  colab_folder_path = "/content/gdrive/MyDrive/NBA/"
  data_file_name_X = "amateurX.pt"
  data_file_name_y = "amateury.pt"
  # save the X so we don't have to recompute it every time
  torch.save(X, colab_folder_path + data_file_name_X)
  # save the y so we don't have to recompute it every time
  torch.save(y, colab_folder_path + data_file_name_y)


In [21]:
X[0, len(X[0]) // 2 :]

array([ 96980.,  25995., 171568., 113721., 185302., 148791., 130692.,
       145731., 182680.,  85785.])

In [22]:
for row in X:
    print("\t".join(map(str, row)))

Streaming output truncated to the last 5000 lines.
104.0	266.0	105.0	51.0	43.0	412.0	7.0	498.0	19.0	133.0	240380.0	144851.0	76147.0	205523.0	66149.0	29768.0	101772.0	135356.0	151427.0	133065.0
21.0	20.0	99.0	64.0	222.0	61.0	157.0	53.0	104.0	114.0	60366.0	17397.0	77359.0	70669.0	49492.0	87996.0	99740.0	21681.0	138792.0	83017.0
412.0	104.0	99.0	81.0	24.0	498.0	117.0	150.0	105.0	120.0	40228.0	222940.0	169110.0	198245.0	136943.0	169289.0	13156.0	158803.0	118159.0	170748.0
412.0	78.0	64.0	245.0	222.0	498.0	59.0	38.0	89.0	58.0	9028.0	16919.0	48179.0	57753.0	67720.0	120432.0	96187.0	60401.0	17899.0	93388.0
86.0	157.0	96.0	117.0	23.0	236.0	122.0	120.0	53.0	103.0	129878.0	107062.0	264789.0	30621.0	171881.0	103794.0	176650.0	154286.0	31531.0	127573.0
143.0	120.0	53.0	236.0	68.0	92.0	121.0	63.0	29.0	4.0	74790.0	121154.0	40840.0	125407.0	122845.0	104934.0	100523.0	28603.0	67347.0	124619.0
91.0	498.0	497.0	39.0	19.0	23.0	22.0	64.0	101.0	37.0	139382.0	164585.0	12591.0	75843.0	156385.0	147356.0	10890

In [23]:
data_file_name_X

'amateurX.pt'

In [24]:
print(len(X))
print(len(y))
print(X[0])
print(len(X[0]))

161064
161064
[1.90000e+01 2.67000e+02 1.19000e+02 1.14000e+02 1.12000e+02 7.20000e+01
 3.00000e+00 1.03000e+02 2.22000e+02 1.61000e+02 9.69800e+04 2.59950e+04
 1.71568e+05 1.13721e+05 1.85302e+05 1.48791e+05 1.30692e+05 1.45731e+05
 1.82680e+05 8.57850e+04]
20


In [25]:
len(X[0])

20

In [26]:
#Ye: We don't need this, because aux info is in X too

# X_stats = stats[['totdmgdealt']].values.astype('float32')
# y_stats = stats['win'].values.astype('float32')

# aux_info = torch.tensor(np.concatenate((X_stats, y_stats.reshape(-1, 1)), axis=1))

# print(aux_info.shape)

#Preprocessing Artificial Dataset

In [ ]:
# Parameters
num_samples = 1000  # Number of games in the dataset

# Initialize lists for X and y
X = []
y = []

# Generate the dataset
for _ in range(num_samples):
    # Generate random numbers for home and away teams
    home_team = np.random.randint(1, 100, size=5)
    away_team = np.random.randint(1, 100, size=5)

    # Calculate variances
    home_variance = np.var(home_team)
    away_variance = np.var(away_team)

    # Determine the winner
    if home_variance > away_variance:
        label = 0  # Away team wins
    else:
        label = 1  # Home team wins

    # Append the data
    X.append(np.concatenate([home_team, away_team]))
    y.append(label)

# Convert lists to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# Print the shape of the tensors
print(f"X shape: {X_tensor.shape}")  # Should be [num_samples, 10]
print(f"y shape: {y_tensor.shape}")  # Should be [num_samples]


X shape: torch.Size([1000, 10])
y shape: torch.Size([1000])


<ipython-input-84-d60c0792877b>:29: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  X_tensor = torch.tensor(X, dtype=torch.float32)


#Preprocessing Pro LOL

In [ ]:
from sklearn.preprocessing import LabelEncoder

if(not load_saved_data):
    df_updated = torch.load("/content/gdrive/MyDrive/2023 research/LoL/merged_df")

    selected_rows = df_updated.loc[[i for n in range(len(df_updated) // 12 + 1) for i in [n * 12 + 10, n * 12 + 11] if i < len(df_updated)]]

    y = selected_rows['result']
    X = selected_rows[['pick1', 'pick2', 'pick3', 'pick4', 'pick5']]
    t1 = []
    t2 = []
    listX = []
    list_true_y = []
    for i, (index, row) in enumerate(X.iterrows()):
      if index % 2 == 0:
        t1_team = X.iloc[i].tolist()
      else:
        t2_team = X.iloc[i].tolist()

        concat_arr = np.concatenate((t1_team, t2_team))
        listX.append(concat_arr.tolist())
        list_true_y.append(y[index])

    flattened_data = [item for sublist in listX for item in sublist]
    # encoder = LabelEncoder()
    # integer_encoded = encoder.fit_transform(flattened_data)

    num_rows = len(listX)
    num_columns = len(listX[0])
    flattened_data = np.array(flattened_data).reshape(num_rows, num_columns)

    X = flattened_data
    true_y = np.array(list_true_y)

    # save the X so we don't have to recompute it every time
    torch.save(X, colab_folder_path + data_file_name_X)
    # save the y so we don't have to recompute it every time
    torch.save(true_y, colab_folder_path + data_file_name_y)


In [ ]:
joblib.dump(encoder, colab_folder_path + encoder_name)

In [ ]:
lolX = X
loly = y

In [ ]:
print(lolX.shape)
print(loly.shape)

In [ ]:
lolX[0]

In [ ]:
X[0][:5]
X[0][5:]

In [ ]:
loly[1]

#Preprocessing Hero Games

In [ ]:
# prompt: read in the CSV file hero_games_fixed_heroes.csv

import pandas as pd

# Assuming 'hero_games_fixed_heroes.csv' is in the current working directory
# If not, provide the correct file path
# file_path = '/content/gdrive/MyDrive/2023 research/NBA/hero_game_dataset.csv'
# file_path = '/content/gdrive/MyDrive/2023 research/NBA/game_dataset_of10.csv'
file_path = '/content/gdrive/MyDrive/NBA/game_dataset_of10.csv'

try:
  df = pd.read_csv(file_path)
  print(df.head())  # Display the first few rows of the DataFrame
except FileNotFoundError:
  print(f"Error: File '{file_path}' not found.")
except pd.errors.ParserError:
    print(f"Error: Could not parse the CSV file '{file_path}'. Check its format.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

                                 team1                                 team2  \
0   [16, 0, 7, 1, 19, 6, 2, 13, 15, 8]   [19, 5, 6, 17, 15, 8, 10, 4, 18, 9]   
1  [13, 1, 17, 3, 9, 5, 12, 11, 15, 6]   [3, 5, 9, 12, 16, 19, 6, 14, 13, 7]   
2  [3, 18, 12, 16, 10, 7, 0, 2, 8, 11]    [3, 1, 13, 18, 11, 6, 19, 7, 5, 9]   
3  [5, 0, 6, 3, 18, 11, 10, 15, 9, 13]    [15, 2, 1, 7, 4, 11, 19, 18, 6, 9]   
4  [16, 17, 1, 2, 13, 7, 0, 3, 19, 12]  [13, 18, 5, 8, 7, 4, 10, 11, 19, 14]   

   outcome  
0        1  
1        2  
2        2  
3        1  
4        2  


In [ ]:
df["team1"]

,team1
0,"[16, 0, 7, 1, 19, 6, 2, 13, 15, 8]"
1,"[13, 1, 17, 3, 9, 5, 12, 11, 15, 6]"
2,"[3, 18, 12, 16, 10, 7, 0, 2, 8, 11]"
3,"[5, 0, 6, 3, 18, 11, 10, 15, 9, 13]"
4,"[16, 17, 1, 2, 13, 7, 0, 3, 19, 12]"
...,...
995,"[8, 7, 5, 4, 14, 6, 3, 19, 0, 10]"
996,"[16, 8, 13, 11, 15, 9, 6, 12, 1, 19]"
997,"[18, 16, 14, 0, 15, 9, 13, 17, 8, 4]"
998,"[16, 7, 0, 8, 19, 3, 9, 2, 13, 17]"


In [ ]:
import pandas as pd
import numpy as np

def combine_teams_no_ast(row):
    team1_str = row['team1'].replace('[', '').replace(']', '').split(',')
    team2_str = row['team2'].replace('[', '').replace(']', '').split(',')

    team1 = [int(x.strip()) for x in team1_str if x.strip()]
    team2 = [int(x.strip()) for x in team2_str if x.strip()]

    return np.concatenate((team1, team2))

X = df.apply(combine_teams_no_ast, axis=1).to_numpy()

In [ ]:
num_players_per_team = 10

In [ ]:
X

array([array([16,  0,  7,  1, 19,  6,  2, 13, 15,  8, 19,  5,  6, 17, 15,  8, 10,
               4, 18,  9])                                                       ,
       array([13,  1, 17,  3,  9,  5, 12, 11, 15,  6,  3,  5,  9, 12, 16, 19,  6,
              14, 13,  7])                                                       ,
       array([ 3, 18, 12, 16, 10,  7,  0,  2,  8, 11,  3,  1, 13, 18, 11,  6, 19,
               7,  5,  9])                                                       ,
       array([ 5,  0,  6,  3, 18, 11, 10, 15,  9, 13, 15,  2,  1,  7,  4, 11, 19,
              18,  6,  9])                                                       ,
       array([16, 17,  1,  2, 13,  7,  0,  3, 19, 12, 13, 18,  5,  8,  7,  4, 10,
              11, 19, 14])                                                       ,
       array([ 8,  6, 14, 11,  7, 15,  3, 19,  1, 18, 13,  2, 16,  8, 18, 11,  4,
              14,  3, 15])                                                       ,
       arr

In [ ]:
X[0]

array([16,  0,  7,  1, 19,  6,  2, 13, 15,  8, 19,  5,  6, 17, 15,  8, 10,
        4, 18,  9])

In [ ]:
X = np.array([np.array(xi) for xi in X])
# prompt: convert X to a new shape of (1000,6)

X = X.reshape(1000, 20)

In [ ]:
X.shape

(1000, 20)

In [ ]:
# prompt: load the df["outcome"] into y

y = df["outcome"].values

In [ ]:
y

array([1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2,
       1, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2,
       1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2,
       2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1,
       1, 2, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2,
       2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2,
       2, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 1,
       1, 1, 2, 1, 2, 2, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2,
       1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 1,
       2, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 1, 1, 2,
       1, 2, 1, 2, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2,
       1, 2, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 2, 1,
       1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1,

In [ ]:
y = y -1

In [ ]:
y

array([0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0,

In [ ]:
y.shape

(1000,)

# Preprocessing Valorant

In [ ]:
import chardet
import pandas as pd

file_path = '/content/gdrive/MyDrive/NBA/Valorant_data.csv'

# Step 1: Detect encoding
with open(file_path, 'rb') as f:
    raw = f.read()
    result = chardet.detect(raw)
    encoding = result['encoding'] if result['encoding'] else 'utf-8'
    print(f"Detected encoding: {encoding} (confidence: {result['confidence']})")

# Step 2: Load into pandas DataFrame with modern syntax
try:
    df = pd.read_csv(file_path, encoding=encoding, on_bad_lines='skip', encoding_errors='replace')
    print(df.head())  # Show first few rows
except Exception as e:
    print(f"Error reading CSV: {e}")


Detected encoding: utf-8 (confidence: 0.99)
   Unnamed: 0   match-datetime  patch    map       team1        team2  \
0           0  2023/4/16 10:00   6.06  Haven  Impulse GW  EGN Esports   
1           1  2023/4/16 10:00   6.06  Haven  Impulse GW  EGN Esports   
2           2  2023/4/16 10:00   6.06  Haven  Impulse GW  EGN Esports   
3           3  2023/4/16 10:00   6.06  Haven  Impulse GW  EGN Esports   
4           4  2023/4/16 10:00   6.06  Haven  Impulse GW  EGN Esports   

   team1-score  team2-score player-name player-team  ... hs-ct   fk  fk-t  \
0           13            6    HolyM0Ly  Impulse GW  ...  0.26  1.0   0.0   
1           13            6      Dolfo   Impulse GW  ...  0.38  0.0   0.0   
2           13            6    zePiCzz   Impulse GW  ...  0.27  2.0   2.0   
3           13            6      s2newb  Impulse GW  ...  0.24  7.0   3.0   
4           13            6    Minikid   Impulse GW  ...  0.80  0.0   0.0   

   fk-ct   fd fd-t fd-ct  fkmd  fkmd-t  fkmd-ct  
0   

<ipython-input-128-853346307>:15: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding=encoding, on_bad_lines='skip', encoding_errors='replace')


In [ ]:
df.columns

Index(['Unnamed: 0', 'match-datetime', 'patch', 'map', 'team1', 'team2',
       'team1-score', 'team2-score', 'player-name', 'player-team', 'agent',
       'rating', 'rating-t', 'rating-ct', 'acs', 'acs-t', 'acs-ct', 'k', 'k-t',
       'k-ct', 'd', 'd-t', 'd-ct', 'a', 'a-t', 'a-ct', 'tkmd', 'tkmd-t',
       'tkmd-ct', 'kast', 'kast-t', 'kast-ct', 'adr', 'adr-t', 'adr-ct', 'hs',
       'hs-t', 'hs-ct', 'fk', 'fk-t', 'fk-ct', 'fd', 'fd-t', 'fd-ct', 'fkmd',
       'fkmd-t', 'fkmd-ct'],
      dtype='object')

In [ ]:
# auxillary MLP columns
aux_columns = ["k", "d", "a", "adr"]
stats = df[aux_columns]
stats.head(10)

,k,d,a,adr
0,20.0,11.0,1.0,160.0
1,16.0,13.0,14.0,158.0
2,12.0,10.0,8.0,148.0
3,18.0,13.0,3.0,158.0
4,8.0,12.0,5.0,64.0
5,11.0,13.0,15.0,124.0
6,14.0,13.0,6.0,130.0
7,14.0,18.0,1.0,111.0
8,11.0,16.0,7.0,93.0
9,9.0,14.0,3.0,87.0


In [ ]:
for col in aux_columns:
    if col not in df.columns:
        raise ValueError(f"Column '{col}' not found in DataFrame.")

In [ ]:
for i, row in df.iterrows():
    team1 = int(row["team1-score"])
    team2 = int(row["team2-score"])

    if team1 > team2:
        print(f"Row {i}: team1 wins ({team1} > {team2})")
    else:
        print(f"Row {i}: team1 does not win ({team1} <= {team2})")


Streaming output truncated to the last 5000 lines.
Row 244459: team1 does not win (12 <= 14)
Row 244460: team1 does not win (12 <= 14)
Row 244461: team1 does not win (12 <= 14)
Row 244462: team1 does not win (12 <= 14)
Row 244463: team1 does not win (12 <= 14)
Row 244464: team1 does not win (12 <= 14)
Row 244465: team1 does not win (12 <= 14)
Row 244466: team1 does not win (12 <= 14)
Row 244467: team1 does not win (12 <= 14)
Row 244468: team1 does not win (12 <= 14)
Row 244469: team1 wins (15 > 13)
Row 244470: team1 wins (15 > 13)
Row 244471: team1 wins (15 > 13)
Row 244472: team1 wins (15 > 13)
Row 244473: team1 wins (15 > 13)
Row 244474: team1 wins (15 > 13)
Row 244475: team1 wins (15 > 13)
Row 244476: team1 wins (15 > 13)
Row 244477: team1 wins (15 > 13)
Row 244478: team1 wins (15 > 13)
Row 244479: team1 does not win (13 <= 15)
Row 244480: team1 does not win (13 <= 15)
Row 244481: team1 does not win (13 <= 15)
Row 244482: team1 does not win (13 <= 15)
Row 244483: team1 does not win 

In [ ]:
df["win"] = (df["team1-score"] > df["team2-score"]).astype(int)

In [ ]:
df["win"]

,win
0,1
1,1
2,1
3,1
4,1
...,...
249454,1
249455,1
249456,1
249457,1


In [ ]:
# Explicitly list all columns to keep
columns_to_keep = ["match-datetime", "team1", "team2", "team1-score", "team2-score", "win", "player-name", "agent"]
df = df[columns_to_keep + aux_columns]

In [ ]:
df.head(30)

,match-datetime,team1,team2,team1-score,team2-score,win,player-name,agent,k,d,a,adr
0,2023/4/16 10:00,Impulse GW,EGN Esports,13,6,1,HolyM0Ly,Killjoy,20.0,11.0,1.0,160.0
1,2023/4/16 10:00,Impulse GW,EGN Esports,13,6,1,Dolfo,Breach,16.0,13.0,14.0,158.0
2,2023/4/16 10:00,Impulse GW,EGN Esports,13,6,1,zePiCzz,Astra,12.0,10.0,8.0,148.0
3,2023/4/16 10:00,Impulse GW,EGN Esports,13,6,1,s2newb,Neon,18.0,13.0,3.0,158.0
4,2023/4/16 10:00,Impulse GW,EGN Esports,13,6,1,Minikid,Sova,8.0,12.0,5.0,64.0
5,2023/4/16 10:00,Impulse GW,EGN Esports,13,6,1,Phatt,Breach,11.0,13.0,15.0,124.0
6,2023/4/16 10:00,Impulse GW,EGN Esports,13,6,1,Dext,Cypher,14.0,13.0,6.0,130.0
7,2023/4/16 10:00,Impulse GW,EGN Esports,13,6,1,fainz,Jett,14.0,18.0,1.0,111.0
8,2023/4/16 10:00,Impulse GW,EGN Esports,13,6,1,al1en,Omen,11.0,16.0,7.0,93.0
9,2023/4/16 10:00,Impulse GW,EGN Esports,13,6,1,silenttt,Sova,9.0,14.0,3.0,87.0


In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

load_saved_data = False
if not load_saved_data:
    X = []
    y = []
    list_X = []
    prev_row = df.iloc[0]
    prev_match_id = prev_row["match-datetime"]

    for i, row in df.iterrows():
        match_id = row["match-datetime"]
        if match_id != prev_match_id:
            if len(list_X) == 2 * num_players_per_team + 2 * num_players_per_team * len(aux_columns):
                # Reorder list_X before appending
                champ_ids = list_X[::(1 + len(aux_columns))]
                aux_values = []
                for j in range(len(aux_columns)):
                    aux_values += list_X[j + 1::(1 + len(aux_columns))]
                listX_reordered = champ_ids + aux_values
                X.append(listX_reordered)
                y.append(prev_row["win"])
            prev_row = row
            prev_match_id = match_id
            list_X = []

        # Add champion ID and aux column values for this player
        list_X.append(row["agent"])
        for col in aux_columns:
            list_X.append(row[col])

    # Append last match
    if len(list_X) == 2 * num_players_per_team + 2 * num_players_per_team * len(aux_columns):
        champ_ids = list_X[::(1 + len(aux_columns))]
        aux_values = []
        for j in range(len(aux_columns)):
            aux_values += list_X[j + 1::(1 + len(aux_columns))]
        listX_reordered = champ_ids + aux_values
        X.append(listX_reordered)
        y.append(prev_row["win"])

    # Flatten and encode if necessary
    flattened_data = [item for sublist in X for item in sublist]

    # Optional: encode if agent IDs are strings or non-numeric
    # encoder = LabelEncoder()
    # integer_encoded = encoder.fit_transform(flattened_data)
    # flattened_data = integer_encoded

    num_rows = len(X)
    num_columns = len(X[0])

    flattened_data = np.array(flattened_data).reshape(num_rows, num_columns)

    X = flattened_data
    y = np.array(y)

    colab_folder_path = "/content/gdrive/MyDrive/NBA/"
    data_file_name_X = "valorantX.pt"
    data_file_name_y = "valoranty.pt"

    torch.save(X, colab_folder_path + data_file_name_X)
    torch.save(y, colab_folder_path + data_file_name_y)


In [ ]:
X[0]

array(['Killjoy', 'Breach', 'Astra', 'Neon', 'Sova', 'Breach', 'Cypher',
       'Jett', 'Omen', 'Sova', '20.0', '16.0', '12.0', '18.0', '8.0',
       '11.0', '14.0', '14.0', '11.0', '9.0', '11.0', '13.0', '10.0',
       '13.0', '12.0', '13.0', '13.0', '18.0', '16.0', '14.0', '1.0',
       '14.0', '8.0', '3.0', '5.0', '15.0', '6.0', '1.0', '7.0', '3.0',
       '160.0', '158.0', '148.0', '158.0', '64.0', '124.0', '130.0',
       '111.0', '93.0', '87.0'], dtype='<U32')

#Preprocessing (SHARED)

##Downsampling

In [27]:
len(X[0])

20

In [28]:
# prompt: get the count of different y values

import collections
y = torch.tensor(y)
y_values = y.flatten().tolist()
counter = collections.Counter(y_values)

print(counter)
print(counter[1]+ counter[0])

Counter({1.0: 82890, 0.0: 78174})
161064


In [29]:
X

array([[1.90000e+01, 2.67000e+02, 1.19000e+02, ..., 1.45731e+05,
        1.82680e+05, 8.57850e+04],
       [1.15000e+02, 6.90000e+01, 1.57000e+02, ..., 1.46754e+05,
        3.57660e+04, 1.17790e+05],
       [1.19000e+02, 4.00000e+01, 9.90000e+01, ..., 1.36987e+05,
        2.63240e+04, 8.15060e+04],
       ...,
       [3.50000e+01, 6.70000e+01, 1.63000e+02, ..., 1.59139e+05,
        3.40010e+04, 1.67443e+05],
       [1.03000e+02, 4.00000e+01, 1.26000e+02, ..., 1.55589e+05,
        1.17671e+05, 1.82770e+04],
       [1.14000e+02, 2.02000e+02, 4.32000e+02, ..., 8.20290e+04,
        5.75630e+04, 1.29580e+04]])

In [30]:
len(X[0])

20

In [31]:
# prompt: downsample y so the labels are the same amount

from imblearn.under_sampling import RandomUnderSampler
#resample only the majority class
rus = RandomUnderSampler(sampling_strategy='majority')

X_train_resampled, y_train_resampled = rus.fit_resample(X, y_values)

y_values = y_train_resampled
counter = collections.Counter(y_values)
X = X_train_resampled
y = y_values
print(counter)


Counter({0.0: 78174, 1.0: 78174})


In [32]:
X[0, len(X[0]) // 2 :]

array([ 96980.,  25995., 171568., 113721., 185302., 148791., 130692.,
       145731., 182680.,  85785.])

In [33]:
# from collections import Counter
# from imblearn.over_sampling import RandomOverSampler
# over_sampler = RandomOverSampler(random_state=42)
# X_res, y_res = over_sampler.fit_resample(X, true_y)
# print(f"Training target statistics: {Counter(y_res)}")
# print(f"Testing target statistics: {Counter(true_y)}")

## Label Encoding X

In [34]:
X[0]

array([1.90000e+01, 2.67000e+02, 1.19000e+02, 1.14000e+02, 1.12000e+02,
       7.20000e+01, 3.00000e+00, 1.03000e+02, 2.22000e+02, 1.61000e+02,
       9.69800e+04, 2.59950e+04, 1.71568e+05, 1.13721e+05, 1.85302e+05,
       1.48791e+05, 1.30692e+05, 1.45731e+05, 1.82680e+05, 8.57850e+04])

In [35]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

aux_len = len_aux_column

# Convert X to a NumPy array if it isn't already
X = np.array(X)
# Split X into comp and aux
comp_X = X[:, :(2 * num_players_per_team)]      # first 10 columns (categorical)
aux_X = X[:, len_aux_column:]       # last 10 columns (numeric)

# Flatten only the comp data for encoding
flattened_comp = comp_X.flatten()

# Apply LabelEncoder only to comp data
encoder = LabelEncoder()
comp_encoded = encoder.fit_transform(flattened_comp)

# Reshape back to original comp shape
comp_encoded = comp_encoded.reshape(comp_X.shape).astype(np.int32)  # keep as int for embedding
aux_X = aux_X.astype(np.float32)  # keep as float for stats processing

# Concatenate comp + aux to form full X
X_encoded = np.hstack([comp_encoded, aux_X])

# Update X and y
X = X_encoded  # mixed int32 + float32 as appropriate
true_y = np.array(y).astype(int)


In [36]:
aux_X

array([[ 96980.,  25995., 171568., ..., 145731., 182680.,  85785.],
       [ 46790.,  62444., 117896., ..., 146754.,  35766., 117790.],
       [ 74879.,  11242.,  66424., ..., 136987.,  26324.,  81506.],
       ...,
       [178243.,  20061., 147074., ..., 110515.,   7187., 111706.],
       [107009., 122588., 151761., ..., 158046.,  91782.,  13818.],
       [ 57542., 188783., 146864., ..., 125717.,  47685., 160599.]],
      dtype=float32)

In [37]:
comp_X.dtype

dtype('float64')

In [38]:
X[0]

array([1.80000e+01, 1.26000e+02, 9.80000e+01, 9.50000e+01, 9.30000e+01,
       6.30000e+01, 2.00000e+00, 8.60000e+01, 1.18000e+02, 1.12000e+02,
       9.69800e+04, 2.59950e+04, 1.71568e+05, 1.13721e+05, 1.85302e+05,
       1.48791e+05, 1.30692e+05, 1.45731e+05, 1.82680e+05, 8.57850e+04])

After checking, we see the team comp data is being reordered/relabelled. The aux data remains identical as before.

In [39]:
aux_X[0]

array([ 96980.,  25995., 171568., 113721., 185302., 148791., 130692.,
       145731., 182680.,  85785.], dtype=float32)

We can invert the comp data back to see if it was matching with the original

In [40]:
encoder.inverse_transform(X[0, :10].astype(int))


array([ 19., 267., 119., 114., 112.,  72.,   3., 103., 222., 161.])

In [41]:
X.shape

(156348, 20)

In [42]:
true_y.shape

(156348,)

In [43]:
len(X[0])

20

If oversampling, we have 18072 samples in total
If undersampling, we have 10360 samples in total

Here for better reliability in data, we chose undersampling.

In [44]:
# prompt: A function that checks whether two rows of X share the same values, but maybe in different order

def rows_have_same_elements_unordered(row1, row2):
  """Checks if two rows have the same two teams, regardless of order.

  Args:
    row1: The first row (list or array).
    row2: The second row (list or array).

  Returns:
    True if the rows have the same two teams, False otherwise.
  """
  return  sorted(row1[0:6]) == sorted(row2[0:6]) and sorted(row1[6:]) == sorted(row2[6:])


In [45]:
# prompt: check on rows in X with identical values

# import numpy as np

# # Find rows with x where rows_have_same_elements_unordered is true
# matching_rows = []
# for i in range(len(X)):
#   for j in range(i+1, len(X)):
#     if rows_have_same_elements_unordered(X[i], X[j]):
#       matching_rows.append((i, j))

# # Print the matching rows
# for row in matching_rows:
#   print(row)


In [46]:
# matching_rows

## Inspection

In [47]:
X[3]

array([6.00000e+00, 1.18000e+02, 1.80000e+01, 6.80000e+01, 1.21000e+02,
       1.70000e+01, 6.90000e+01, 1.00000e+01, 1.34000e+02, 1.03000e+02,
       1.28787e+05, 1.51221e+05, 1.47647e+05, 1.43555e+05, 6.81900e+04,
       1.47393e+05, 1.09306e+05, 2.12727e+05, 2.04090e+04, 1.06663e+05])

In [48]:
X[167]

array([1.12000e+02, 3.00000e+00, 1.05000e+02, 1.01000e+02, 4.70000e+01,
       8.60000e+01, 6.00000e+01, 1.28000e+02, 1.00000e+02, 8.00000e+01,
       7.34020e+04, 1.41775e+05, 9.82720e+04, 1.05735e+05, 1.17396e+05,
       1.61883e+05, 1.23009e+05, 2.09010e+04, 1.43464e+05, 1.34933e+05])

In [49]:
##TODO
## take out those rows, sum them up in y

In [50]:

# save the X so we don't have to recompute it every time
torch.save(X, colab_folder_path + data_file_name_X)
# save the y so we don't have to recompute it every time
torch.save(true_y, colab_folder_path + data_file_name_y)

In [51]:
# prompt: save the encoder

import joblib

# Assuming 'encoder' is your LabelEncoder object
joblib.dump(encoder, colab_folder_path + encoder_name)

['/content/gdrive/MyDrive/NBA/amateur_encoder.joblib']

#Model

In [52]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn

from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=True)

In [54]:
models = {
    'Decision Tree' : DecisionTreeClassifier(),
    'Neural Network' : MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(8, 8), random_state=1),
    'KNN' : KNeighborsClassifier(n_neighbors=3)
}

# results = {}

def evaluate_model(model, X_train, y_train, X_test, y_test):
    # Train the model
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate evaluation metrics
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    win_per = percent_wins(y_pred)
    print(y_pred)

    return recall, precision, f1, accuracy, win_per

# iterate through all models
for name, model in models.items():
    #DEBUG ONLY comment out this below to run the baselines
    break
    # Evaluate model, calculate metrics and store them
    recall, precision, f1, accuracy, win_per = evaluate_model(model, X_train, y_train, X_test, y_test)

    print(name)
    print(f"Recall: {recall}")
    print(f"Precision: {precision}")
    print(f"F1-Score: {f1}")
    print(f"Accuracy: {accuracy}")
    print(f"Percentage of tensors with value 1: {win_per:.2f}%")
    print()



## pass hidden size as arg? (see linked stackoverflow in issue w/ loss)

In [55]:
vocab_size = len(np.unique(X[:,: 2 * num_players_per_team]))
print(vocab_size)

136


### Baseline NN

In [56]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, embedding_dim):
        super(Net, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim) # Assuming vocab size is 200000
        self.ln1 = nn.Linear(embedding_dim * num_players_per_team * 2, 16)  # Changed in_features to embedding_dim * 10 = 80
        self.ln2 = nn.Linear(16, 8)
        self.ln3 = nn.Linear(8, 1)


    def forward(self, x):
        x = self.embedding(x)
        x = x.view(x.size(0), -1) # Flatten the embedding output

        x = self.ln1(x)
        x = F.relu(x)
        x = self.ln2(x)
        x = F.relu(x)
        x = self.ln3(x)
        x = torch.sigmoid(x) # Applying sigmoid for binary classification
        return x



```
# This is formatted as code
```

### Siamese Network

In [57]:
# prompt: use the above model to create a siamese network

class SiameseNetwork(nn.Module):
  def __init__(self, embedding_dim):
    super(SiameseNetwork, self).__init__()
    self.embedding = nn.Embedding(num_embeddings= vocab_size, embedding_dim=embedding_dim)
    self.ln1 = nn.Linear(embedding_dim * num_players_per_team, 16)
    self.ln2 = nn.Linear(16, 16)
    self.ln3 = nn.Linear(16, 1)
    ##TODO:: dropout layers, maybe

  def forward(self, x):
    # split x into x1 and x2
    x1, x2 = x[:, :(len(X[0])//2)], x[:, (len(X[0])//2):]
    # Pass the input through the embedding layer
    x1 = self.embedding(x1)
    x2 = self.embedding(x2)
    # Reshape the output of the embedding layer to a 2D tensor
    x1 = x1.view(x1.size(0), -1)
    x2 = x2.view(x2.size(0), -1)
    x1 = self.ln1(x1)
    x1 = F.relu(x1)
    x2 = self.ln1(x2)
    x2 = F.relu(x2)
    x1 = self.ln2(x1)
    x1 = F.relu(x1)
    x2 = self.ln2(x2)
    x2 = F.relu(x2)
    x1 = self.ln3(x1)
    x1 = F.relu(x1)
    x2 = self.ln3(x2)
    x2 = F.relu(x2)
    # ##0 for left side win, 1 for right side win
    output1 = torch.sigmoid(x1)
    output2 = torch.sigmoid(x2)
    # IMPORTANT:: right now output win/lose ratio, can be any fraction between 0 and 1, not necessarily 1/0 win/lose.
    output1 = x1
    output2 = x2
    output = torch.sigmoid(output1 - output2)

    return output


### Transformer



In [58]:
# prompt: A transformer that treats each value in every row of X as a token, outputs a single value
# a standard transformer
class TransformerModel(nn.Module):
    def __init__(self, embedding_dim, input_dim = vocab_size,
                 num_heads = 2, num_layers = 1, hidden_dim = 64, output_dim = 1):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        #encoder layer but no masking applied
        encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_dim,
                                                   nhead=num_heads,
                                                   dim_feedforward=hidden_dim,
                                                   batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(embedding_dim, output_dim)

    def forward(self, x):
        # x begins with shape (batch_size, seq_len)
        x = self.embedding(x)
        # x  with shape (batch_size, seq_len, embedding_dim)
        x = x.permute(1, 0, 2)  # Reshape for Transformer input (seq_len, batch_size, embedding_dim)
        x = self.transformer_encoder(x)
        x = x.mean(dim=0)  # Average the embeddings across the sequence
        x = self.fc(x)
        return torch.sigmoid(x)

    def get_embedding(self, x):
        return self.embedding(x)

# Example usage:
# input_dim = len(unique_values_in_X)  # Number of unique tokens in X
# embedding_dim = 16
# num_heads = 2
# num_layers = 2
# hidden_dim = 32
# output_dim = 1

# model = TransformerModel(input_dim, embedding_dim, num_heads, num_layers, hidden_dim, output_dim)

# Assuming X_train is a tensor of shape (batch_size, seq_len)
# output = model(X_train)


### Auxillary MLP (multilayer perceptron)

In [59]:
# prompt: A secondary neural network that take in a feature

class Aux_MLP(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=64, embedding_dim=64, dropout=0.2):
        super(Aux_MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_dim, embedding_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        x = torch.sigmoid(x)
        return x

### Siamese + Auxillary MLP

In [60]:
# prompt: a siamese auxillary MLP
## Possible issure here in how
class ModelB(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim = 1):
    super(ModelB, self).__init__()
    self.input_dim = input_dim

    self.fc1 = nn.Linear(input_dim, hidden_dim*4 )
    self.fc2 = nn.Linear(hidden_dim*4 , hidden_dim)
    self.fc3 = nn.Linear(hidden_dim, output_dim)
    self.relu = nn.ReLU()
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    # print("x.shape:", x.shape)

    ## note: this x input here will be auxillary information only, no team comp info
    # split x into x1 and x2
    x1, x2 = x[:, :self.input_dim], x[:, self.input_dim:]

    # print("Before fc1:", x1.shape)
    # print("fc1: ", self.fc1)
    # print("Before fc1:", x2.shape)

    x1 = self.fc1(x1)
    x1 = self.relu(x1)
    x1 = self.fc2(x1)
    x1 = self.relu(x1)
    x1 = self.fc3(x1)
    x1 = self.sigmoid(x1)

    # print("After fc1:", x1.shape)
    # print("After fc1:", x2.shape)

    # print("x1 nan after fc1:", torch.isnan(x1).any().item())
    # print("x2 nan after fc1:", torch.isnan(x2).any().item())

    x2 = self.fc1(x2)
    x2 = self.relu(x2)
    x2 = self.fc2(x2)
    x2 = self.relu(x2)
    x2 = self.fc3(x2)
    x2 = self.sigmoid(x2)

    # print("x1 nan after fc2:", torch.isnan(x1).any().item())
    # print("x2 nan after fc2:", torch.isnan(x2).any().item())

    # print("x1 nan after sigmoid:", torch.isnan(x1).any().item())
    # print("x2 nan after sigmoid:", torch.isnan(x2).any().item())

    output = torch.sigmoid(x1 - x2)
    # print("output nan:", torch.isnan(output).any().item())
    return output

            # class TeamAuxMLP(nn.Module):
#     def __init__(self, vocab_size, embedding_dim=32, hidden_dim=64):
#         super().__init__()
#         self.embedding = nn.Embedding(vocab_size, embedding_dim)
#         self.encoder = nn.Sequential(
#             nn.Linear(embedding_dim * num_players_per_team, hidden_dim),
#             nn.ReLU(),
#             nn.Linear(hidden_dim, hidden_dim // 2),
#             nn.ReLU()
#         )

#     def forward(self, x):
#         x = self.embedding(x)
#         x = x.view(x.size(0), -1)
#         return self.encoder(x)

# class SiameseAuxModel(nn.Module):
#     def __init__(self, vocab_size, embedding_dim=32, hidden_dim=64):
#         super().__init__()
#         self.team_net = TeamAuxMLP(vocab_size, embedding_dim, hidden_dim)
#         self.classifier = nn.Sequential(
#             nn.Linear(hidden_dim, hidden_dim // 2),
#             nn.ReLU(),
#             nn.Linear(hidden_dim // 2, 1)
#         )

#     def forward(self, x):
#         mid = x.size(1) // 2
#         team1 = x[:, :mid]
#         team2 = x[:, mid:]
#         t1 = self.team_net(team1)
#         t2 = self.team_net(team2)
#         combined = torch.cat([t1, t2], dim=1)
#         return torch.sigmoid(self.classifier(combined))


### Siamese + Transformer

In [61]:
# prompt: A transformer that treats each value in every row of X as a token, outputs a single value

class SiamTransformerModel(nn.Module):
    def __init__(self, embedding_dim, input_dim = vocab_size,
                 num_heads = 2, num_layers = 1, hidden_dim = 64, output_dim = 1):
        super(SiamTransformerModel, self).__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        #encoder layer but no masking applied
        encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=num_heads, dim_feedforward=hidden_dim)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(embedding_dim, output_dim)

    def forward(self, x):
        # split x into x1 and x2
        x1, x2 = x[:, :num_players_per_team], x[:,num_players_per_team : 2 * num_players_per_team]
        # print("1", x1)

        x1 = self.embedding(x1)
        x1 = x1.permute(1, 0, 2)  # Reshape for Transformer input (seq_len, batch_size, embedding_dim)
        # print("2", x1)
        x1 = self.transformer_encoder(x1)
        # print("3", x1)
        x1 = x1.mean(dim=0)  # Average the embeddings across the sequence
        x1 = self.fc(x1)
        x1 = torch.sigmoid(x1)
        # print("4", x1)

        x2 = self.embedding(x2)
        x2 = x2.permute(1, 0, 2)  # Reshape for Transformer input (seq_len, batch_size, embedding_dim)
        x2 = self.transformer_encoder(x2)
        x2 = x2.mean(dim=0)  # Average the embeddings across the sequence
        x2 = self.fc(x2)
        x2 = torch.sigmoid(x2)
        # print("5", x2)
        output = torch.sigmoid(x1 - x2)
        return output

    def get_embedding(self, x):
        return self.embedding(x)



###Combine A+B

We treat the siamese transformer as model A and the Siamese Aux MLP as model B

In [62]:
X[0].shape

(20,)

In [63]:
# prompt: the combined models of model A (Siamese Transformer) and model B (Siamese Auxillary MLP)
class ModelAnB(nn.Module):
  def __init__(self, transformer_model, aux_model, transformer_output_dim=1):
        super(ModelAnB, self).__init__()
        # include models
        self.transformer = transformer_model
        self.aux_model = aux_model
        # Combine both outputs
        self.final_fc = nn.Linear(transformer_output_dim, 1)
        self.aux_weight = nn.Parameter(torch.tensor(0.5))
        #force the weight to be 1 or 0 to turn on/off model A or B.
        #self.aux_weight = nn.Tensor([1])
        #self.aux_weight = nn.Tensor([0])


  def forward(self, X):
    if isinstance(X, list):
        X = torch.tensor(X, dtype=torch.float32).to(next(self.parameters()).device)
    X_comp = X[:, :(len(X[0])//2)]
    X_aux = X[:, (len(X[0])//2):]

    # print("team_comp shape:", X_comp.shape)
    # print("aux_info shape:", X_aux.shape)

    # Transformer processes team comp
    comp_output = self.transformer(X_comp.int())
    # ModelB processes aux features
    aux_output = self.aux_model(X_aux)

    # print("comp_output shape:", X_comp.shape)
    # print("aux_output shape:", X_aux.shape)

    # print("comp_output:", comp_output)
    # print("aux_output:", aux_output)

     # idea1: Combine via addition maybe add weight to outputs for better results
    combined = (1 - self.aux_weight) * comp_output + self.aux_weight * aux_output

    # print("combined shape:", combined.shape)
    # print("combined:", combined)
    # idea2: Combine via concat and pass through a final linear layer idk which would be best
    # combined = torch.cat([comp_output, aux_output], dim=1)  # shape: (batch_size, 2)

    combined = self.final_fc(combined)
    return torch.sigmoid(combined)


# class ModelAnB(nn.Module):
#     """
#     Hybrid model that merges outputs from:
#     - Model A: Siamese Transformer (team composition)
#     - Model B: Auxiliary MLP (auxiliary team info)

#     Supports:
#     - Weighted addition of outputs
#     - Optional concatenation + projection
#     - Adjustable model weighting (aux_weight)
#     """
#     def __init__(self, transformer_model, aux_model, transformer_output_dim=1,
#                  use_concat=False, learnable_weight=True, init_weight=0.5):
#         super(ModelAnB, self).__init__()
#         self.transformer = transformer_model
#         self.aux_model = aux_model
#         self.use_concat = use_concat

#         if use_concat:
#             self.fusion = nn.Sequential(
#                 nn.Linear(transformer_output_dim * 2, transformer_output_dim),
#                 nn.ReLU(),
#                 nn.Linear(transformer_output_dim, 1)
#             )
#         else:
#             self.fusion = nn.Linear(transformer_output_dim, 1)

#         if learnable_weight:
#             self.aux_weight = nn.Parameter(torch.tensor(init_weight))
#         else:
#             self.register_buffer('aux_weight', torch.tensor(init_weight))

#     def forward(self, team_comp, aux_info):
#         comp_output = self.transformer(team_comp)
#         aux_output = self.aux_model(aux_info)

#         if self.use_concat:
#             # Combine by concatenating outputs and projecting
#             combined = torch.cat([comp_output, aux_output], dim=1)
#             out = self.fusion(combined)
#         else:
#             # Combine by weighted addition of outputs
#             combined = (1 - self.aux_weight) * comp_output + self.aux_weight * aux_output
#             out = self.fusion(combined)

#         return torch.sigmoid(out)




### issue with loss? (fixed in oversampling)
https://stackoverflow.com/questions/65219569/pytorch-gives-incorrect-results-due-to-broadcasting

Code example

```
# Define the model
model = NeuralNetwork()

# Define the loss function
loss_function = CrossEntropyLoss()

# Define the optimizer
optimizer = GradientDescentOptimizer()

# Train the model
for epoch in range(200):
    # Get the data
    data = get_data()

    # Calculate the loss
    loss = loss_function(model(data), labels)

    # X and true_y

    # Backpropagate the loss
    gradient = loss_function.backward()

    # Update the parameters
    optimizer.update(parameters, gradient)

    print("loss: "+ str(loss))
# Evaluate the model
accuracy = model.evaluate(data, labels)
```

#Split Training and Testing

Can we split based on season? (season feature in the dataset)

In [64]:
embedding_dim = 8

# model = Net
# model_name += "NN.pth"

# model = SiameseNetwork
# model_name += "SiameseNetwork.pth"

# model = TransformerModel
# model_name += "TransformerModel.pth"

# model = Aux_MLP
# model_name += "Aux_MLP.pth"

# model = SiamTransformerModel
# model_name += "SiamTransformerModel.pth"

# Instantiate your base models
input_dim = X.shape[1] // 2
hidden_dim = 64

##TODO:: personally I would set the output dim to be not 1. But let's make this work first, and then try to make it not 1.

# Create base models
siamese_transformer = SiamTransformerModel(embedding_dim=embedding_dim)
aux_info = ModelB(input_dim=len_aux_column//2, hidden_dim=hidden_dim)

# Instantiate the combined model
model = ModelAnB(transformer_model=siamese_transformer, aux_model=aux_info)
model_name += "combinedA+B.pth"


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [65]:
model_name

'amateur_modelcombinedA+B.pth'

In [66]:
print("model:", model)
print("type(model):", type(model))
print("embedding_dim:", embedding_dim)
print("type(embedding_dim):", type(embedding_dim))

if(load_saved_data):
  #net = model(embedding_dim).to(device)

  # use below for combined A+B
  net = model.to(device)

  # net = torch.load(colab_folder_path + model_name)
  #net.load_state_dict(torch.load(colab_folder_path + model_name))
  #net.eval()
print(torch.cuda.is_available())
  #encoder = joblib.load(colab_folder_path + encoder_name)

model: ModelAnB(
  (transformer): SiamTransformerModel(
    (embedding): Embedding(136, 8)
    (transformer_encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=8, out_features=8, bias=True)
          )
          (linear1): Linear(in_features=8, out_features=64, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=64, out_features=8, bias=True)
          (norm1): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (fc): Linear(in_features=8, out_features=1, bias=True)
  )
  (aux_model): ModelB(
    (fc1): Linear(in_features=5, out_features=256, bias=True)
    (fc2): Linear(in_features=25

##Testing code here

So I can debug the code and inspect the values

In [67]:
report_cycle = 100

In [68]:
import numpy as np

y = np.array(y)

In [69]:
from torch.utils.data import DataLoader, TensorDataset

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

k_fold = KFold(n_splits=10, shuffle=True, random_state=123)
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
team_comp = []
aux_info = []
criterion = torch.nn.BCELoss()


for train_index, test_index in k_fold.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42, shuffle=True)

    print(f"TensorDataset is: {TensorDataset} (type: {type(TensorDataset)})")
    print(type(X_val), type(y_val))
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32)
    X_val = torch.tensor(X_val, dtype=torch.float32)
    y_val = torch.tensor(y_val, dtype=torch.float32)

    train_dataset = TensorDataset(X_train, y_train)
    val_dataset = TensorDataset(X_val, y_val)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    # net = model(embedding_dim).to(device)

    net = model.to(device)

    optimizer = optim.Adam(net.parameters(), lr=0.00001)
    patience = 100
    best_loss = float('inf')
    counter = 0

    best_model_path = "best_model.pt"

    for epoch in range(10000):
        net.train()
        for X_batch, y_batch in train_loader:
            # Move data to GPU
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            pred_y = net(X_batch)

            loss = criterion(pred_y.flatten(), y_batch.float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if epoch % report_cycle == 0:
            print(f"Epoch {epoch} training loss: {loss.item()}")

        net.eval()
        val_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                # Move data to GPU
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)

                val_pred_y = net(X_batch)
                val_loss += criterion(val_pred_y.flatten(), y_batch.float()).item()
        val_loss /= len(val_loader)

        if epoch % report_cycle == 0:
            print(f"Validation loss: {val_loss}")

        if val_loss < best_loss:
            best_loss = val_loss
            counter = 0
            torch.save(net.state_dict(), best_model_path)
        else:
            counter += 1
            if counter >= patience:
                print(f"Early stopping on epoch: {epoch}")
                break
    # Load the best model for evaluation
    net.load_state_dict(torch.load(best_model_path))
    net.eval()
    with torch.no_grad():
        # Move test data to GPU
        X_test, y_test = X_test.to(device), y_test.to(device)

        pred_y = net(X_test)
        pred_y[pred_y > 0.5] = 1
        pred_y[pred_y <= 0.5] = 0
        acc = torch.sum(pred_y.flatten() == y_test) / len(y_test)
        # print(f"Accuracy: {acc}")
        accuracy_scores.append(acc)

        # Convert to numpy arrays for metric calculations
        y_test_np = y_test.cpu().numpy()
        pred_y_np = pred_y.flatten().cpu().numpy()

        precision = precision_score(y_test_np, pred_y_np)
        recall = recall_score(y_test_np, pred_y_np)
        f1 = f1_score(y_test_np, pred_y_np)

        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)

        print(f"Accuracy: {acc}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1-score: {f1}")

    #right now only running one 10-fold, comment to run ten 10-folds.
    # break

# avg_accuracy = np.mean(accuracy_scores.cpu())
# avg_precision = np.mean(precision_scores.cpu())
# avg_recall = np.mean(recall_scores.cpu())
# avg_f1 = np.mean(f1_scores.cpu())

# print("\n--- Average Metrics Across Folds ---")
# print(f"Average Accuracy: {avg_accuracy}")
# print(f"Average Precision: {avg_precision}")
# print(f"Average Recall: {avg_recall}")
# print(f"Average F1-score: {avg_f1}")

Using device: cpu
TensorDataset is: <class 'torch.utils.data.dataset.TensorDataset'> (type: <class 'type'>)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
Epoch 0 training loss: 0.7132587432861328
Validation loss: 0.6950751242312518


KeyboardInterrupt: 

In [70]:
# prompt: count how many pred_y are 1 or 0

count_ones = (pred_y == 1).sum().item()
count_zeros = (pred_y == 0).sum().item()

print(f"Number of 1s in pred_y: {count_ones}")
print(f"Number of 0s in pred_y: {count_zeros}")


Number of 1s in pred_y: 0
Number of 0s in pred_y: 0


In [71]:
# prompt: count how many pred_y are 1 or 0

count_ones = (y_test == 1).sum().item()
count_zeros = (y_test == 0).sum().item()

print(f"Number of 1s in y_test: {count_ones}")
print(f"Number of 0s in y_test: {count_zeros}")


Number of 1s in y_test: 7703
Number of 0s in y_test: 7932


In [72]:
import pandas as pd

# Convert tensors to numpy arrays (detach from computation graph first)
pred_y_np = pred_y.detach().cpu().numpy().flatten()
y_test_np = y_test.detach().cpu().numpy().flatten()

# Align lengths to the smaller of the two
min_len = min(len(pred_y_np), len(y_test_np))
pred_y_np = pred_y_np[:min_len]
y_test_np = y_test_np[:min_len]

# Create a DataFrame for comparison
comparison_df = pd.DataFrame({
    'pred_y': pred_y_np,
    'y_test': y_test_np
})

# Display the DataFrame
print(comparison_df)

AttributeError: 'numpy.ndarray' object has no attribute 'detach'

In [73]:
import pandas as pd

# Detach and convert tensors to NumPy arrays
pred_y_np = pred_y.detach().cpu().numpy().flatten()
y_test_np = y_test.detach().cpu().numpy().flatten()

# Create a DataFrame for side-by-side comparison
comparison_df = pd.DataFrame({'pred_y': pred_y_np, 'y_test': y_test_np})

# Display the first and last 40 rows (if available)
n = 40
if len(comparison_df) > 2 * n:
    display_df = pd.concat([comparison_df.head(n), comparison_df.tail(n)])
else:
    display_df = comparison_df  # show entire DataFrame if too small

print(display_df)


AttributeError: 'numpy.ndarray' object has no attribute 'detach'

In [74]:
net

ModelAnB(
  (transformer): SiamTransformerModel(
    (embedding): Embedding(136, 8)
    (transformer_encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=8, out_features=8, bias=True)
          )
          (linear1): Linear(in_features=8, out_features=64, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=64, out_features=8, bias=True)
          (norm1): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (fc): Linear(in_features=8, out_features=1, bias=True)
  )
  (aux_model): ModelB(
    (fc1): Linear(in_features=5, out_features=256, bias=True)
    (fc2): Linear(in_features=256, out_

In [75]:
unique_values_in_X

NameError: name 'unique_values_in_X' is not defined

In [76]:
X_train

tensor([[7.0000e+00, 1.7000e+01, 8.0000e+01,  ..., 1.5074e+05, 1.2732e+05,
         1.5187e+05],
        [1.2800e+02, 0.0000e+00, 1.2000e+02,  ..., 2.5757e+05, 5.3844e+04,
         2.4418e+05],
        [7.1000e+01, 3.9000e+01, 5.4000e+01,  ..., 1.5976e+05, 9.1314e+04,
         1.1653e+04],
        ...,
        [2.8000e+01, 0.0000e+00, 9.7000e+01,  ..., 1.9086e+04, 1.4067e+05,
         1.1895e+05],
        [2.0000e+01, 7.1000e+01, 1.1000e+02,  ..., 2.5051e+05, 6.4200e+04,
         6.6294e+04],
        [1.5000e+01, 6.0000e+00, 4.7000e+01,  ..., 1.2252e+05, 1.2782e+05,
         1.8523e+04]])

In [77]:
accuracy_scores

[]

In [78]:
recall_scores

[]

In [ ]:
# Convert PyTorch tensors in the lists to Python floats
accuracy_scores = [score.item() if isinstance(score, torch.Tensor) else score for score in accuracy_scores]
precision_scores = [score.item() if isinstance(score, torch.Tensor) else score for score in precision_scores]
recall_scores = [score.item() if isinstance(score, torch.Tensor) else score for score in recall_scores]
f1_scores = [score.item() if isinstance(score, torch.Tensor) else score for score in f1_scores]

# Calculate averages using NumPy
avg_accuracy = np.mean(accuracy_scores)
avg_precision = np.mean(precision_scores)
avg_recall = np.mean(recall_scores)
avg_f1 = np.mean(f1_scores)

# Print average metrics
print("\n--- Average Metrics Across Folds ---")
print(f"Average Accuracy: {avg_accuracy}")
print(f"Average Precision: {avg_precision}")
print(f"Average Recall: {avg_recall}")
print(f"Average F1-score: {avg_f1}")

In [ ]:
# prompt: same as above but taking out 0 values and corresponding values from the lists accuracy_scores precision_scores recall_scores f1_scores

import numpy as np

# Assuming accuracy_scores, precision_scores, recall_scores, and f1_scores are lists of numerical values

# Find indices where any of the scores is 0
zero_indices = [i for i, (acc, prec, rec, f1) in enumerate(zip(accuracy_scores, precision_scores, recall_scores, f1_scores)) if acc == 0 or prec == 0 or rec == 0 or f1 == 0]


# Remove elements at zero_indices from all lists
accuracy_scores = [score for i, score in enumerate(accuracy_scores) if i not in zero_indices]
precision_scores = [score for i, score in enumerate(precision_scores) if i not in zero_indices]
recall_scores = [score for i, score in enumerate(recall_scores) if i not in zero_indices]
f1_scores = [score for i, score in enumerate(f1_scores) if i not in zero_indices]

# Convert lists to numpy arrays for easier calculations (if necessary)
accuracy_scores = np.array(accuracy_scores)
precision_scores = np.array(precision_scores)
recall_scores = np.array(recall_scores)
f1_scores = np.array(f1_scores)


# Calculate and print averages
avg_accuracy = np.mean(accuracy_scores) if len(accuracy_scores)>0 else 0
avg_precision = np.mean(precision_scores) if len(precision_scores)>0 else 0
avg_recall = np.mean(recall_scores) if len(recall_scores)>0 else 0
avg_f1 = np.mean(f1_scores) if len(f1_scores)>0 else 0

print("\n--- Average Metrics Across Folds (excluding zero values)---")
print(f"Average Accuracy: {avg_accuracy}")
print(f"Average Precision: {avg_precision}")
print(f"Average Recall: {avg_recall}")
print(f"Average F1-score: {avg_f1}")

### Save the model

You don't need to save the encoder here. The label encoder is set up early on, never trained, so it is saved early too.

In [ ]:
torch.save(net.state_dict(), colab_folder_path + model_name)

## Embedding inspection

In [ ]:
net.embedding

In [ ]:
# prompt: use net's embedding layer to extract the embedding of X[0]

# Assuming 'net' is your trained model and 'X' is your data
embedding = net.embedding(torch.tensor(X[0]).to(device))
print(encoder.inverse_transform(X[0]))
embedding

In [ ]:
embedding = net.embedding(torch.tensor(X[1]).to(device))
print(encoder.inverse_transform(X[1]))
embedding

In [ ]:
name_to_embedding = {}
for unique_X in unique_values_in_X:
  embedding = net.embedding(torch.tensor(unique_X).to(device))
  name_to_embedding[encoder.inverse_transform([unique_X])[0]] = embedding

In [ ]:
unique_values_in_X

In [ ]:
name_to_embedding

In [ ]:
# Joey's spaghetti code
# get embeddings for all NBA players
# name_to_embedding = {}
# for game in X:
#   embeds = net.embedding(torch.tensor(game).to(device))
#   names = encoder.inverse_transform(game)
#   for i in range(len(names)):
#     name_to_embedding[names[i]] = embeds[i]

In [ ]:
print(len(name_to_embedding))

In [ ]:
name_to_embedding[135]

In [ ]:
## takes two n-dimensional vectors, and calculates the manhattan distance between the two vectors
def manhattan_distance(x1, x2):
    if(len(x1) != len(x2)):
        print("Error: x1 and x2 must be the same length")
    dist = 0
    for i in range(len(x1)):
        diff = x1[i] - x2[i]
        dist += abs(diff)
    return dist

In [ ]:
def nearest_embeddings(name):
  if name not in name_to_embedding:
    print("Error: name not in name_to_embedding")
    return
  closest = []
  name_embedding = name_to_embedding[name]
  for name, embedding in name_to_embedding.items():
    #print(name, manhattan_distance(name_embedding, embedding))
    closest.append((name, manhattan_distance(name_embedding, embedding)))
  closest.sort(key=lambda x: x[1])
  return closest[:10]

In [ ]:
nearest_embeddings(99)

In [ ]:
nearest_embeddings(44)

In [ ]:
nearest_embeddings(37)

In [ ]:
nearest_embeddings("Yao Ming")

In [ ]:
nearest_embeddings("Kobe Bryant")

In [ ]:
nearest_embeddings("Shaquille O'Neal")

In [ ]:
nearest_embeddings("Paul George")

In [ ]:
nearest_embeddings("Calbert Cheaney")

A even better idea is to analysis their key, query, value separately. However, this is not easy in the current implementation. You will need to have custom transformer encoder to do that.

#Real Train Test Experiments

In [ ]:
def run_model(model, X, y, embedding_dim):
  #X = torch.tensor(X, dtype=torch.long)  # Use torch.long for indices
  #y = torch.tensor(y, dtype=torch.float) # Use torch.float for labels
  k_fold = KFold(n_splits=10, shuffle=True, random_state=123)
  # Create a list to store the accuracy scores for each fold
  accuracy_scores = []
  # criterion = torch.nn.MSELoss()
  criterion = torch.nn.BCELoss()
  for train_index, test_index in k_fold.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    #split training data for validation
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42, shuffle=True)


    # Minibatching Implementation
    from torch.utils.data import TensorDataset, DataLoader

    # Convert NumPy arrays to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32)
    X_val = torch.tensor(X_val, dtype=torch.float32)
    y_val = torch.tensor(y_val, dtype=torch.float32)

    # Create TensorDatasets (paired samples and labels)
    train_dataset = TensorDataset(X_train, y_train)
    val_dataset = TensorDataset(X_val, y_val)

    # Create DataLoaders for minibatching
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


    # initialize the net
    net = model(embedding_dim)  # Training
    # net = SiameseNetwork(embedding_dim) # Siamese

    optimizer = optim.Adam(net.parameters(), lr= 0.0001)
    patience= 100
    best_loss = float('inf')
    counter = 0
    for epoch in range(1000):
        ##batch training
        #TODO try minibatch
        #pred_y = net(X_train.long())

      net.train()  # Set model to training mode
    for X_batch, y_batch in train_loader:
        # Move batch to device (CPU or GPU)
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Forward pass
        pred_y = net(X_batch)

        # Compute loss for this batch
        loss = criterion(pred_y.flatten(), y_batch)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()



        # Zero the gradients
        optimizer.zero_grad()

        # Backpropagate the loss
        loss.backward()

        # Update the parameters
        optimizer.step()

        if(epoch%100 == 0):
          print("Epoch "+ str(epoch) +" training loss: "+ str(loss.item()))
        #comment out the two lines below to make the task more difficult
        #X_val = X_test
        #y_val = y_test

        net.eval()  # Set model to evaluation mode
        val_loss = 0

        # No gradient tracking needed during evaluation
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)

                # Forward pass
                val_pred_y = net(X_batch)

                # Accumulate loss across batches
                val_loss += criterion(val_pred_y.flatten(), y_batch).item()

        # Average loss across all validation batches
        val_loss /= len(val_loader)

        print("validation loss: "+ str(val_loss.item()))

        if val_loss.item() < best_loss:
            best_loss = val_loss.item()
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping on epoch: " + str(epoch))
                break





    # testing
    pred_y = net(X_test)
    #convert all pred_y value more than 0.5 to 1,
    #and the rest to 0
    pred_y[pred_y>0.5] = 1
    pred_y[pred_y<=0.5] = 0
    # Accuracy on the training set.

    acc = torch.sum(pred_y.flatten() == y_test) / len(y_test)
    print("Accuracy:", acc)
    # Add the accuracy score to the list
    accuracy_scores.append(acc)

  print("Average accuracy:", np.mean(accuracy_scores))

I tried the following combinations of learning parameters



In [ ]:
class SiameseNetwork(nn.Module):
    def __init__(self, embedding_dim):
        super(SiameseNetwork, self).__init__()
        self.embedding = nn.Embedding(200000, embedding_dim)

        # The input to the linear layer is the embedding dimension (embedding_dim)
        # multiplied by the length of the input sequence. Assuming the input
        # sequence has 5 words, and embedding_dim is 8, the input size would be 40 (8*5).
        # Change `in_features` to reflect the actual size of the input to the linear layer.
        self.ln1 = nn.Linear(embedding_dim * 5, 16)  # Adjust in_features to match embedding_dim * sequence_length
        self.ln2 = nn.Linear(16, 8)
        self.ln3 = nn.Linear(8, 1)

    def forward(self, x):
        x1 = x[:5]
        x2 = x[5:]
        x1 = self.embedding(x1)
        x2 = self.embedding(x2)

        # Reshape to (batch_size, embedding_dim * sequence_length)
        x1 = x1.view(x1.size(0), -1)
        x2 = x2.view(x2.size(0), -1)

        x1 = self.ln1(x1)
        x1 = F.relu(x1)
        x2 = self.ln1(x2)
        x2 = F.relu(x2)
        x = torch.abs(x1 - x2)
        x = self.ln2(x)
        x = F.relu(x)
        x = self.ln3(x)
        x = torch.sigmoid(x)
        return x

In [ ]:
run_model(Net, lolX, loly, 8)

NameError: name 'lolX' is not defined

In [ ]:
run_model(SiameseNetwork, X, y, 8)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (5x160 and 40x16)

In [ ]:
run_model(TransformerModel, lolX, loly, 8)

NameError: name 'lolX' is not defined

#END OF FILE

Use the following code to salvage on the output if google disconnects you when your colab takes too long.

In [ ]:
# prompt: read the new 2.txt file into log_output

log_output = ""
with open("2.txt", "r") as file:
    log_output = file.read()
log_output

FileNotFoundError: [Errno 2] No such file or directory: '2.txt'

In [ ]:
import re
import numpy as np



# Regular expressions to capture the scores
accuracy_pattern = re.compile(r"Accuracy: ([\d.]+)")
precision_pattern = re.compile(r"Precision: ([\d.]+)")
recall_pattern = re.compile(r"Recall: ([\d.]+)")
f1_score_pattern = re.compile(r"F1-score: ([\d.]+)")

# Extract scores
accuracies = accuracy_pattern.findall(log_output)
precisions = precision_pattern.findall(log_output)
recalls = recall_pattern.findall(log_output)
f1_scores = f1_score_pattern.findall(log_output)

# Print the results
for i in range(len(accuracies)):
    print(f"Run {i+1}:")
    print(f"  Accuracy: {accuracies[i]}")
    print(f"  Precision: {precisions[i]}")
    print(f"  Recall: {recalls[i]}")
    print(f"  F1-score: {f1_scores[i]}")

#convert accuracies to list of number
accuracies = [float(acc) for acc in accuracies]
precisions = [float(prec) for prec in precisions]
recalls = [float(rec) for rec in recalls]
f1_scores = [float(f1) for f1 in f1_scores]

print( "Average Accuracy:", {np.mean(accuracies)})
print( "Average Precision:", {np.mean(precisions)})
print( "Average Recall:", {np.mean(recalls)})
print( "Average F1-score:", {np.mean(f1_scores)})

Average Accuracy: {np.float64(nan)}
Average Precision: {np.float64(nan)}
Average Recall: {np.float64(nan)}
Average F1-score: {np.float64(nan)}


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
